In [1]:
import os
from data.source.pg_experiment import get_pg_experiment_dataset, AUDIO_PATH
import polars as pl
%load_ext autoreload
%autoreload 1
%aimport models.SimpleCNN_v2
from models.SimpleCNN_v2 import PronunciationDataset, collate_fn, SimpleCNN, train, evaluate
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb
import wandb_config

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

wandb: Currently logged in as: fischbach-kamil (fischbach-kamil-pg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
df_pron, _ = get_pg_experiment_dataset(".wav")

df_stageI_polish = df_pron.filter(pl.col("word_id").str.starts_with("a") & pl.col("mother")
    .str.strip_chars()
    .str.to_lowercase()
    .is_in(["polish", "polski"]))

df_other_lang = df_pron.filter(
    pl.col("word_id").str.starts_with("a") &
    (~pl.col("mother")
       .str.strip_chars()
       .str.to_lowercase()
       .is_in(["polish", "polski"]))
)


In [3]:
count_missing = 0
rec_paths = df_other_lang.select("rec_path").to_series()

for path in rec_paths:
    full_path = os.path.normpath(os.path.join(AUDIO_PATH, path))
    if not os.path.exists(full_path):
        count_missing += 1


count_missing_pl = 0
rec_paths = df_stageI_polish.select("rec_path").to_series()

for path in rec_paths:
    full_path = os.path.normpath(os.path.join(AUDIO_PATH, path))
    if not os.path.exists(full_path):
        count_missing_pl += 1

print(f"Missing files in mother language Polish: {count_missing_pl}")
print(f"Missing files in mother language other than Polish: {count_missing}")

Missing files in mother language Polish: 0
Missing files in mother language other than Polish: 2


In [4]:
# Find indices of rows with missing files
rec_paths = df_other_lang.select("rec_path").to_series()
missing_indices = []

for i, path in enumerate(rec_paths):
    full_path = os.path.normpath(os.path.join(AUDIO_PATH, path))
    if not os.path.exists(full_path):
        missing_indices.append(i)

# Print the deleted rows
deleted_rows = df_other_lang[missing_indices]
print(deleted_rows)

df_other_lang = df_other_lang.filter(
    ~pl.Series(range(len(df_other_lang))).is_in(pl.Series(missing_indices).implode())
)



shape: (2, 7)
┌────────────┬───────┬─────────┬───────────────────┬───────────┬────────┬─────────┐
│ id_student ┆ value ┆ word_id ┆ rec_path          ┆ univ      ┆ gender ┆ mother  │
│ ---        ┆ ---   ┆ ---     ┆ ---               ┆ ---       ┆ ---    ┆ ---     │
│ i64        ┆ i64   ┆ str     ┆ str               ┆ str       ┆ str    ┆ str     │
╞════════════╪═══════╪═════════╪═══════════════════╪═══════════╪════════╪═════════╡
│ 757        ┆ 1     ┆ a0      ┆ stageI\757\a0.wav ┆ CLES_UMK2 ┆ f      ┆ chinese │
│ 757        ┆ 1     ┆ a1      ┆ stageI\757\a1.wav ┆ CLES_UMK2 ┆ f      ┆ chinese │
└────────────┴───────┴─────────┴───────────────────┴───────────┴────────┴─────────┘


In [5]:
import polars as pl
import numpy as np

train_split = 0.6
val_split = 0.2
test_split = 1 - train_split - val_split
def stratified_split(df: pl.DataFrame, label_col: str, train_frac=train_split, val_frac=val_split, seed=42):
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [6]:
# dataset = PronunciationDataset(df_stageI_polish, base_dir=AUDIO_PATH)
dataset_other = PronunciationDataset(df_other_lang ,base_dir=AUDIO_PATH)

In [7]:
train_pl, val_pl, test_pl = stratified_split(df_stageI_polish, label_col="value", train_frac=0.6, val_frac=0.2)

dataset_train = PronunciationDataset(train_pl, base_dir=AUDIO_PATH)
dataset_val = PronunciationDataset(val_pl, base_dir=AUDIO_PATH)
dataset_test = PronunciationDataset(test_pl, base_dir=AUDIO_PATH)


In [8]:
from pytorch_dataloader import MemoryLoadedDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=4)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)

train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")

Loaded train loader into memory
Loaded validation loader into memory


In [44]:
model = SimpleCNN()
model.eval()

SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.15, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (dropout3): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [45]:
# Model variables definition.
pth = "SimpleCNN_v2_wDecay_batch_dropout.pth"
lr = 5e-4  # Reduce from 1e-3
epochs = 100
model = SimpleCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)  # Add L2 regularization
# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)
criterion = nn.BCELoss()

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the run
    name="SimpleCNN-v2-run-3",
    config={
        "Name": 'SimpleCNN-v2',
        "learning_rate": lr ,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "SimpleCNN-v2",
        "dataset": "Stage-I-only-polish",
        "train_val_test(%)": f'{train_split}-{val_split}-{test_split}',
        "epochs": epochs,
    },
)

# Training loop
best_val_loss = float('inf')
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    # Update learning rate
    scheduler.step(val_loss)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("simplecnn-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets send to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.7006, Train Acc: 0.5679, Val Loss: 0.6798, Val Acc: 0.6003
Epoch 2, Train Loss: 0.6695, Train Acc: 0.5865, Val Loss: 0.6656, Val Acc: 0.5890
Epoch 3, Train Loss: 0.6567, Train Acc: 0.6019, Val Loss: 0.6547, Val Acc: 0.6056
Epoch 4, Train Loss: 0.6530, Train Acc: 0.5996, Val Loss: 0.6519, Val Acc: 0.6082
Epoch 5, Train Loss: 0.6495, Train Acc: 0.5999, Val Loss: 0.6638, Val Acc: 0.5812
Epoch 6, Train Loss: 0.6466, Train Acc: 0.6002, Val Loss: 0.6538, Val Acc: 0.6012
Epoch 7, Train Loss: 0.6474, Train Acc: 0.5885, Val Loss: 0.6519, Val Acc: 0.6047
Epoch 8, Train Loss: 0.6413, Train Acc: 0.6095, Val Loss: 0.6421, Val Acc: 0.6065
Epoch 9, Train Loss: 0.6370, Train Acc: 0.6092, Val Loss: 0.6443, Val Acc: 0.6073
Epoch 10, Train Loss: 0.6330, Train Acc: 0.6208, Val Loss: 0.6447, Val Acc: 0.6091
Epoch 11, Train Loss: 0.6314, Train Acc: 0.6162, Val Loss: 0.6500, Val Acc: 0.6091
Epoch 12, Train Loss: 0.6269, Train Acc: 0.6176, Val Loss: 0.6420, Val Acc: 0.6161
Epoch 13, Tra

train_acc,▁▃▃▃▃▄▅▅▅▅▆▆▆▆▇▇▇▇█▇▇▇█▇▇███████████████
train_loss,██▇▇▇▇▆▆▅▄▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▂▁▂▁▁▁▁▁▂▁▂
val_acc,▃▁▃▃▃▄▄▄▄▅▅▆▅▆▆▆▆▇▇▇▇██▇████████████████
val_loss,▃▄▃▂▂▁▂▂▂▃▁▁▂▁▁▂▄▅▆▆▇▇▇▇▇█▇████▇████████
model_eval,SimpleCNN( (conv1)...
train_acc,0.73626
train_loss,0.48548
val_acc,0.65969
val_loss,0.68861


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_saved = SimpleCNN()
model_saved.load_state_dict(torch.load(os.path.join(RESULT_DIRECTORY, "initial_model_t&val_080101split.pth")))
model_saved.to(device)
model_saved.eval()

RuntimeError: Error(s) in loading state_dict for SimpleCNN:
	Missing key(s) in state_dict: "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var". 